Tatoebaコーパスをダウンロードし、前処理するところを自分の手でやりたい場合は下のセルを実行する。

ただし、前処理済みのデータセットが既に用意されているので、必ずしも実行する必要はない。

In [1]:
import os

os.chdir("/")
!git clone https://github.com/hir777/Machine_Translation_Proto.git
!pip install sacremoses
!pip install spacy
!pip install tqdm
!pip install datasets
!pip install mecab-python3
!pip install unidic
!python3 -m unidic download
!pip install argparse

Cloning into 'Machine_Translation_Proto'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 422 (delta 128), reused 189 (delta 100), pack-reused 170
Receiving objects: 100% (422/422), 9.70 MiB | 3.19 MiB/s, done.
Resolving deltas: 100% (206/206), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 32.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ef428603a408b9e2dcdc15e7e60cfb51386f053686da04d5c673538dee506d84
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/publi

In [2]:
import os
os.chdir("/")
!git clone https://github.com/hir777/Machine_Translation_Proto.git

fatal: destination path 'Machine_Translation_Proto' already exists and is not an empty directory.


In [3]:
os.chdir("/Machine_Translation_Proto/")
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
os.chdir("/Machine_Translation_Proto/corpus/src")

!python3 create_dataset.py \ 
--repo_path /Machine_Translation_Proto/ \
--len_filter \
--overlap_filter \
--ratio_filter

usage: create_dataset.py [-h] [--repo_path REPO_PATH] [--len_filter]
                         [--overlap_filter] [--ratio_filter]
create_dataset.py: error: unrecognized arguments:  


NameError: ignored

In [4]:
os.chdir("/Machine_Translation_Proto/")
!bash pre-process.sh REPO_PATH="/Machine_Translation_Proto"

+ for ARGUMENT in "$@"
++ cut -f1 -d=
++ echo REPO_PATH=/Machine_Translation_Proto
+ KEY=REPO_PATH
+ KEY_LENGTH=9
+ VALUE=/Machine_Translation_Proto
+ export REPO_PATH=/Machine_Translation_Proto
+ REPO_PATH=/Machine_Translation_Proto
+ TRAIN_EN=/Machine_Translation_Proto/corpus/data/train.en
+ TRAIN_JA=/Machine_Translation_Proto/corpus/data/train.ja
+ VALID_EN=/Machine_Translation_Proto/corpus/data/valid.en
+ VALID_JA=/Machine_Translation_Proto/corpus/data/valid.ja
+ TEST_EN=/Machine_Translation_Proto/corpus/data/test.en
+ TRAIN_SP=/Machine_Translation_Proto/src/train_sp.py
+ ENCODE=/Machine_Translation_Proto/src/encode.py
+ cat /Machine_Translation_Proto/corpus/data/train.en /Machine_Translation_Proto/corpus/data/train.ja
+ python /Machine_Translation_Proto/src/train_sp.py --input train.enja --prefix bpe --vocab_size 4000 --character_coverage 0.9995
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: train.enja
  input_format: 
  model_prefix: bpe
  

In [5]:
!wc -l train.ja
!wc -l train.en

152784 train.ja
152784 train.en


In [5]:
!fairseq-train \
    /Machine_Translation_Proto/data-bin \
    --fp16 \
    --save-interval 5 \
    --log-interval 1 \
    --log-format simple \
    --max-epoch 30 \
    --update-freq 1 \
    --max-update 30000 \
    --max-tokens 4000 \
    --arch bart_base \
    --encoder-normalize-before \
    --decoder-normalize-before \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 4096 \
    --encoder-attention-heads 8 \
    --encoder-layers 8 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 4096 \
    --decoder-attention-heads 8 \
    --decoder-layers 8 \
    --share-all-embeddings \
    --dropout 0.3 \
    --attention-dropout 0.0 \
    --activation-dropout 0.0 \
    --activation-fn gelu \
    --optimizer adam \
    --adam-betas '(0.9, 0.999)' \
    --lr 0.0015 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 4000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 1.0 \
    --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.3 \
    | tee train.log

Streaming output truncated to the last 5000 lines.
2022-06-09 01:29:45 | INFO | train_inner | epoch 022:    432 / 598 loss=5.283, nll_loss=1.554, ppl=2.94, wps=14856, ups=3.71, wpb=4000, bsz=200, num_updates=12988, lr=0.000832435, gnorm=0.258, clip=0, loss_scale=32, train_wall=0, wall=3878
2022-06-09 01:29:45 | INFO | train_inner | epoch 022:    433 / 598 loss=5.284, nll_loss=1.535, ppl=2.9, wps=11188.9, ups=3.22, wpb=3474, bsz=200, num_updates=12989, lr=0.000832403, gnorm=0.277, clip=0, loss_scale=32, train_wall=0, wall=3878
2022-06-09 01:29:45 | INFO | train_inner | epoch 022:    434 / 598 loss=5.376, nll_loss=1.72, ppl=3.29, wps=13299, ups=3.62, wpb=3674, bsz=496, num_updates=12990, lr=0.00083237, gnorm=0.224, clip=0, loss_scale=32, train_wall=0, wall=3878
2022-06-09 01:29:46 | INFO | train_inner | epoch 022:    435 / 598 loss=5.194, nll_loss=1.438, ppl=2.71, wps=14083.8, ups=3.58, wpb=3936, bsz=328, num_updates=12991, lr=0.000832338, gnorm=0.173, clip=0, loss_scale=32, train_wall=0

In [6]:
!fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint_last.pt \
    --beam 1 \
    --lenpen 0.6 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu /Machine_Translation_Proto/corpus/data/test.ja

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 33.8 61.3/39.8/27.1/19.8 (BP = 1.000 ratio = 1.002 hyp_len = 196043 ref_len = 195748)


In [7]:
!head output.txt /Machine_Translation_Proto/corpus/data/test.ja

==> output.txt <==
 謎 を 解く の に 明らか な 説明 は し なかっ た 。
 カフェ で お しゃべり しよう と し て いる 。
 マラソン は 揺れる だろう 。 雨降り か 晴れ て いる 。
 彼 は お 金 を 返し て く
 駅 まで どの くらい かかり ます か 。
 我々 は 11 時 以降 の 話 を し た 。
 <unk> を 飛ばす こと が 危険 な こと も ある 。
 その 切符 で 100 ユーロ 足 が 出る 。
 彼 は 自分 の 書い た 曲 を 女の子 に 演じ た 。
 メグ は 多く の 学校 に 通っ て いる 。

==> /Machine_Translation_Proto/corpus/data/test.ja <==
私 たち は その なぞ の は
カフェイン の 摂取 量 を 減らそう と し て いる ん です 。 
マラソン は 晴雨 に かかわら ず 開か れ ます 。 
彼 は 間違い なく 金 を 返す と 約束 し た 。 
あなた が 駅 に 着く の に どの くらい かかり ます か 。 
私 たち は 11 時 すぎ まで 話し た 。 
凧 を 揚げる の は 危険 か も しれ ない 。 
切符 は 百 ユーロ です 。 
彼 は その 少女 の ため に 自作 の 曲 を 一 曲 弾い た 。 
メグ は 学校 の いろいろ な 行事 に 参加 する 。 


In [8]:
from src.translate import en2ja
os.chdir("/Machine_Translation_Proto/src/")
en_sent = input('英文を入力 > ')

ja = en2ja(checkpoint_dir="/Machine_Translation_Proto/checkpoints/",
           checkpoint_file="checkpoint30.pt",
           data_name_or_path="/Machine_Translation_Proto/data-bin/",
           path_bpe_model="/Machine_Translation_Proto/bpe.model",
           en=str(en_sent))
print(ja)

英文を入力 > I have a pen
私はペンを1本持って
